In [1]:
from reg_utils import *
from preprocessing import crop_image
import numpy as np


ModuleNotFoundError: No module named 'cv2'

In [2]:
dim = (192,240)

image = Image.open('2B4B65304845774E7852364E334935366D34745863513D3D_537153536F422F464D673568615A614E722B565347774455364A6367436A4C48_20100406_1.png')
image.show()

data = np.array(image)

cropped = crop_image(data,dim)

Image.fromarray(cropped).show()


NameError: name 'cv2' is not defined

In [2]:
dim = (192, 240)
name = get_filename(dim, rgb=True, coord=True, flip=True, noise=2, settype='trainX', setsize=0.8)
print(name)

192x240_3channel_coord_flip_noise2_trainX0.8


In [2]:
from PIL import Image, ImageOps
import numpy as np
from os import listdir

In [29]:
from PIL import Image, ImageDraw

im = Image.new('L', (350, 500), (0))
draw = ImageDraw.Draw(im)

draw.ellipse((250, 250, 315, 285), fill=(255))

im.show()



In [ ]:
#test PIL

from PIL import Image

print("Image info:")
# load the image
image = Image.open('68344674535469533878377A584E43593337513070413D3D_537153536F422F464D67357430426C43547431456F774455364A6367436A4C48_20110406_4.png')
# summarize some details about the image
print(image.format)
print(image.mode)
print(image.size)

print()
print("Mask info:")
# load the image
image = Image.open('68344674535469533878377A584E43593337513070413D3D_537153536F422F464D67357430426C43547431456F774455364A6367436A4C48_20110406_4_mask.png')
# summarize some details about the image
print(image.format)
print(image.mode)
print(image.size)
img_grey= image.convert('RGB')
print(img_grey.mode)


In [11]:
#manual data augmentation (rotation)


def load_data(nr_samples = 438, folder_path = 'priors_all_copy', 
              rgb = False, dim = (192, 240), rotate = True, mirror = True):
    input_images = []
    target_pixels = []
    passed_pix = 0
    passed_img = 0
    rot = 1
    for filename in sorted(listdir(folder_path)):
        if "DS_Store" in filename:
            pass
        elif "mask" not in filename:
            try:
                #open image
                img_data = Image.open(folder_path + '/' + filename)
                if rgb == True:
                    img_rgb = img_data.convert('RGB')
                else: 
                    img_rgb = img_data
                #open mask
                maskname = filename.split('.')[0] + "_mask.png"
                pix_data = Image.open(folder_path + '/' + maskname)
                if rotate == True: 
                    rot = 2
                for i in range(rot):
                    #rotate image + add
                    rot_img = img_rgb.rotate(i*180, Image.NEAREST, expand = 1)
                    img_resized = rot_img.resize(dim)
                    data = np.array(img_resized)
                    new_filename = filename.split('.')[0] + "_" + str(i) + ".png"
                    input_images.append([new_filename,data])
                    
                    #rotate mask + add
                    rot_pix = pix_data.rotate(i*180, Image.NEAREST, expand = 1)
                    data = np.array(rot_pix)
                    pixel_coord = np.where(data == True)
                    img_size = rot_pix.size
                    new_maskname = filename.split('.')[0] + "_mask_" + str(i) + ".png"
                    target_pixels.append([new_maskname, (pixel_coord[1][0]/img_size[0])*dim[0], (pixel_coord[0][0]/img_size[1])*dim[1]])
                    
                    if mirror:
                        #mirror + add
                        orig_mir = ImageOps.mirror(rot_img)
                        img_resized = orig_mir.resize(dim)
                        data = np.array(img_resized)
                        new_filename = filename.split('.')[0] + "_m_" + str(i) + ".png"
                        input_images.append([new_filename,data])

                        #mirror mask + add                  
                        mir_pix = ImageOps.mirror(rot_pix)
                        data = np.array(mir_pix)
                        pixel_coord = np.where(data == True)
                        img_size = rot_pix.size
                        new_maskname = filename.split('.')[0] + "_mask_m_" + str(i) + ".png"
                        target_pixels.append([new_maskname, (pixel_coord[1][0]/img_size[0])*dim[0], (pixel_coord[0][0]/img_size[1])*dim[1]])

                    
            except (UnidentifiedImageError, NameError):
                passed_img += 1

    input_images = np.asarray(input_images)
    target_pixels = np.asarray(target_pixels)
    
    return input_images, target_pixels


In [12]:
def gen_dataset(input_images, target_pixels): #generate dataset
    
    #remove filenames (only keep input and target)
    X_img = np.squeeze(input_images[:,1])
    Y_pix = target_pixels[:,[1,2]]
    
    #shuffle same way
    rng_state = np.random.get_state()
    np.random.shuffle(X_img)
    np.random.set_state(rng_state)
    np.random.shuffle(Y_pix)
    
    X_img = np.stack(X_img, axis=0)
    
    X_img = X_img/255

    
    return X_img, Y_pix

In [13]:
def divide_train_test(X_img, Y_pix, trainsize, rgb=False):
    
    # divide into train and test:
    split = int(len(X_img)*trainsize)
    X_train = X_img[0:split]
    X_test = X_img[split+1:-1]
    Y_train = Y_pix[0:split]
    Y_test = Y_pix[split+1:-1]
    
    # format input arrays
    if rgb == False:
        X_train = X_train[:,:,:,np.newaxis]
        X_test = X_test[:,:,:,np.newaxis]
        
    return X_train, Y_train, X_test, Y_test

In [14]:
rgb = False
#dim = (192, 240) #width, heigth (ratio 0.8) 
dim = (384, 480)
#dim = (960, 1200) #största testad hittills
#dim = (1920, 2400)
#dim = (2400, 3000) #största möjliga
    
input_images, target_pixels = load_data(rgb = rgb, dim = dim, rotate=False, mirror=False)

X_img, Y_pix = gen_dataset(input_images, target_pixels)

X_train, Y_train, X_test, Y_test = divide_train_test(X_img, Y_pix, trainsize = 0.8)

In [15]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
#print(Y_test)

(1401, 480, 384, 1)
(1401, 2)
(349, 480, 384, 1)
(349, 2)


In [18]:
from numpy import savetxt
# save to csv file
X_filename = 'X_'+ str(dim[0]) + 'x' + str(dim[1]) + '_' + 'rgb-' + str(rgb) + '.csv'
Y_filename = 'Y_'+ str(dim[0]) + 'x' + str(dim[1]) + '_' + 'rgb-' + str(rgb) + '.csv'
print(X_filename)
print(Y_filename)

savetxt(X_filename, X_img, delimiter=',')#, fmt='%s')
savetxt(Y_filename, Y_pix, delimiter=',')#, fmt='%s')

X_384x480_rgb-False.csv
Y_384x480_rgb-False.csv


ValueError: Expected 1D or 2D array, got 3D array instead